In [422]:
import numpy as np
import pandas as pd
import Levenshtein

In [423]:
pd.options.display.max_rows = 600
pd.options.display.max_columns = 85

In [424]:
df = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Old Files dump/new_totalprojects.csv')
xlsx = pd.ExcelFile('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/Housing/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)
df1_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/HOME_2007-2017_fixed_address.csv')
df2_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/CDBG_2007-2017_fixed_address.csv')
df1_fixed = df1_fixed.drop('Unnamed: 0', axis = 1, errors='ignore')
df2_fixed = df2_fixed.drop('Unnamed: 0', axis = 1, errors='ignore')
df_fixed = pd.concat([df1_fixed, df2_fixed])
df_fixed.index = range(850)
df_truth = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/TestDB/Ship/Project_v02.csv')

In [425]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [426]:
fixed = df_fixed['Address']
truth = df_truth['Address']

In [427]:
fixed[fixed.str.contains("APT")]

98              2515 BRIERWOOD DR APT 209
132              2010 W BROAD AVE APT 124
133                2200 CHAMPAGNE DRAPT C
135           525 DON CUTLER SR DR APT 38
138               2006 LAFAYETTE CR APT C
140                 2527 HILLTOP DR APT 2
141                  2533 HILLTOP DRAPT 5
142                 1308 HOBSON ST APT 40
144                  901 LEISURE CT APT D
159       1404 WHISPERING PINES RD APT Q2
160       1404 WHISPERING PINES RD APT N3
172              2010 W BROAD AVE APT 109
177              2206 HABERSHAM RD APT 58
182              320 S JACKSON ST APT 566
185               2010 W BROAD AVE APT 87
186               2010 W BROAD AVE APT 61
187               2010 W BROAD AVE APT 88
188                 215 BROADWAY ST APT 4
192               2206 HABERSHAM RDAPT 25
196             1336 MERCANTILE DR APT 57
197              2016 LAFAYETTE CIR APT D
198                  901 LEISURE CT APT D
199    2368 MEADOWLARK COMMONS CT APT 202
204           509 N WESTOVE BLVD A

In [428]:
fixed.iloc[118] = '1112 A PEACHTREE TER'
fixed.iloc[121] = '305 A STATION CROSSING'
fixed.iloc[130] = '2517 B CARDINAL DR'
fixed.iloc[173] = '2201 C CHAMPAGNE DR'
fixed.iloc[175] = '2200 D CHAMPAGNE DR'
fixed.iloc[206] = '2200 C CHAMPAGNE LN'
fixed.iloc[226] = '3201 A FERNRIDGE DR'
fixed.iloc[233] = '2316 B GLENEAGLES DR'
fixed.iloc[133] = '2200 CHAMPAGNE DR APT C'
fixed.iloc[141] = '2533 HILLTOP DR APT 5'
fixed.iloc[192] = '2206 HABERSHAM RD APT 25'
fixed.iloc[225] = '525 DON CUTLER SR DR APT 110'
fixed.iloc[235] = '320 S JACKSON ST APT 10'

In [429]:
number_streets = {'1ST', '2ND', '3RD', '4TH', '5TH', '6TH', '7TH', '8TH', '9TH',
              '10TH', '11TH', '12TH', '13TH', '14TH', '15TH', '16TH', '17TH',
              '18TH', '19TH', '20TH'}

In [430]:
fixed_addresses = []
mh = []
for i in range(len(fixed)):
    address = fixed.iloc[i]
    split_address = address.split()
    new_address = ""
    contains_mh = False
    for j in range(1,len(split_address)):
        word = split_address[j]
        if hasNumbers(word) and word not in number_streets:
            if not contains_mh:
                mh.append(word)
            contains_mh = True
        elif len(word) == 1:
            if word != 'N' and word != 'W' and word != 'S' and word != 'E':
                if not contains_mh:
                    mh.append(word)
                contains_mh = True
            else:
                new_address += word + " "
        else:
            if not contains_mh:
                contains_mh=True
                mh.append("NA")
            new_address += split_address[j] + " "
    fixed_addresses.append((split_address[0] + " " + new_address).strip())
mh = pd.Series(mh)
fixed_addresses = pd.Series(fixed_addresses)

In [431]:
dic = {} 
for i in range(850):
    fixedaddr = fixed_addresses.iloc[i]
    truthaddr = truth.iloc[i]
    if fixedaddr != truthaddr:
        dic[fixedaddr] = truthaddr

In [434]:
dic = {'1118 WHITNEY AVE': '1118 W WHITNEY AVE',
 '1403 GREENBRIAR CT': '1403 GREENBRIER CT',
 '1602 ESTELLE AVE': '1602 ESTELLE ST',
 '1330 MOBILE RD': '1330 MOBILE AVE',
 '504 MAGNOLIA ST': '504 S MAGNOLIA ST',
 '1110 WADDELL AVE': '1110 W WADDELL AVE',
 '2226 OLD DOMINION DR': '2226 OLD DOMINION RD',
 '2701 DONCASTER RD': '2701 DONCASTER DR',
 '2515 BRIERWOOD DR APT': '2515 BRIERWOOD DR',
 '323 INDEPENDENCE AVE': '323 INDEPENDENCE DR',
 '2010 LAFAYETTE CR': '2010 LAFAYETTE CIR',
 '909 HIGHLAND AVE': '909 W HIGHLAND AVE',
 '603 N HALEY ST': '603 HALEY ST',
 '305 STATION CROSSING': '305 STATION CROSSING DR',
 '2517 CARDINAL DR': '2517 CARDINAL ST',
 '2010 W BROAD AVE APT': '2010 W BROAD AVE',
 '2200 CHAMPAGNE DR APT': '2200 CHAMPAGNE DR',
 '525 DON CUTLER SR DR APT': '525 DON CUTLER SR DR',
 '1902 FLINTLOCK DR': '1902 S FLINTLOCK DR',
 '2006 LAFAYETTE CR APT': '2006 LAFAYETTE CIR',
 '2527 HILLTOP DR APT': '2527 HILLTOP DR',
 '2533 HILLTOP DR APT': '2533 HILLTOP DR',
 '1308 HOBSON ST APT': '1308 HOBSON ST',
 '427 KINGSWOOD DR': '427 KINGSWOOD CT',
 '901 LEISURE CT APT': '901 LEISURE CT',
 '2501 REDWOOD CR UNIT': '2501 REDWOOD CT',
 '1404 WHISPERING PINES RD APT': '1404 WHISPERING PINES RD',
 '933 RANDOLPH DR': '933 RANDOLPH ST',
 '826 FERRELL CT': '826 FARRELL CT',
 '2206 HABERSHAM RD APT': '2206 HABERSHAM RD',
 '320 S JACKSON ST APT': '320 S JACKSON ST',
 '215 BROADWAY ST APT': '215 N BROADWAY ST',
 '1005 E FOURTH AVE': '1005 E 4TH AVE',
 '1336 MERCANTILE DR APT': '1336 MERCANTILE DR',
 '2016 LAFAYETTE CIR APT': '2016 LAFAYETTE CIR',
 '2368 MEADOWLARK COMMONS CT APT': '2368 MEADOWLARK COMMONS CT',
 '509 N WESTOVE BLVD APT': '509 N WESTOVER BLVD',
 '509 N WESTOVER BLVD APT': '509 N WESTOVER BLVD',
 '2200 CHAMPAGNE LN': '2200 CHAMPAGNE DR',
 '1904 ELKHORN LN APT': '1904 ELKHORN LN',
 '2202 LIONEL LN APT': '2202 LIONEL LN',
 '1700 SHOREHAM DR APT': '1700 SHOREHAM DR',
 '2000 TOMPKINS AVE APT': '2000 TOMPKINS AVE',
 '1019 UNIVERSITY DR APT': '1019 UNIVERSITY ST',
 '509 WESTOVER BLVD APT': '509 N WESTOVER BLVD',
 '2415 DAWSON RD APT': '2415 DAWSON RD',
 '1200 KINGSTOWN CT APT': '1200 KINGSTOWN CT',
 '2415 BRIERWOOD DR APT': '2415 BRIERWOOD DR',
 '2312 GLENEAGLE DR APT': '2312 GLENEAGLES DR',
 '1624 MARYLAND DR APT': '1624 MARYLAND DR',
 '215 N BROADWAY ST APT': '215 N BROADWAY ST',
 '1901 BUCK LN APT': '1901 BUCK LN',
 '810 PINE AVE APT': '810 PINE AVE',
 '2732 MCCLAIN LN': '2732 MCLAIN LN',
 '421 S WESTOVER BLVD APT': '421 S WESTOVER BLVD',
 '1605 SCHLEY AVE APT': '1605 SCHLEY AVE',
 '4000 GILLIONVILLE RD APT': '4000 GILLIONVILLE RD',
 '2710 W OAKRDIGE DR': '2710 W OAKRIDGE DR',
 '1404 WHISPERING PINES AVE APT': '1404 WHISPERING PINES RD',
 '321 STATION CROSSING DR APT': '321 STATION CROSSING DR',
 '1015 AMSTERDAM DR': '1015 AMSTERDAM LN',
 '3202 FERNRIDGE DR APT': '3202 FERNRIDGE DR',
 '2312 GLENEAGLES DR APT': '2312 GLENEAGLES DR',
 '313 STATION CROSSING DR APT': '313 STATION CROSSING DR',
 '200 S VALENCIA DR': '200 S VALENCIA ST',
 '1117 WADDELL AVE': '1117 W WADDELL AVE',
 '1316 E FOURTH AVE': '1316 E 4TH AVE',
 '1419 HIGHLAND AVE': '1419 W HIGHLAND AVE',
 '1712 HIGHLAND AVE': '1712 W HIGHLAND AVE',
 '1925 RIVERVIEW DR': '1925 S RIVERVIEW CIR',
 '2002 SKYLANE DR': '2002 N SKYLANE DR',
 '305 SHADOWLAWN DR': '305 S SHADOWLAWN DR',
 '823 SHADOWLAWN': '823 S SHADOWLAWN DR',
 '1400 WHITNEY AVE': '1400 W WHITNEY AVE',
 '307 SHADOWLAWN DR': '307 S SHADOWLAWN DR',
 '706 EASTTOWN RD': '706 E TOWN RD',
 '105 HILLSIDE AVE': '105 HILLSIDE DR',
 '1918 HIGHLAND AVE': '1918 W HIGHLAND AVE',
 '2207 OXFORD DR': '2207 OXFORD RD',
 '700 COLQUITT CR': '700 COLQUITT CIR',
 '701 WILLIARD AVE': '701 WILLARD AVE',
 '425 W SOCIETY AVE': '106 N MONROE ST',
 '302 S CASON AVE': '302 CASON ST',
 '1105 JORDAN AVE': '1105 JORDAN ST',
 '1120 LINCOLN AVE': '1120 W LINCOLN AVE',
 '1400 ESTELLE AVE': '1400 ESTELLE ST',
 '1401 E CAMPBELL CARROLL ST': '1401 E CAMPBELL ST/108 CARROLL ST',
 '1501 WADDELL AVE': '1501 W WADDELL AVE',
 '1907 GROVELAND DR': '1907 GROVELAND RD',
 '1914 HIGHLAND AVE': '1914 W HIGHLAND AVE',
 '2016 MLK JR DR': '2016 MARTIN LUTHER KING JR DR',
 '219 TALAFAX ST': '219 TALAFAX RD',
 '2206 HABERSHAM RD APTS': '2206 HABERSHAM RD',
 '303 INDEPENDENCE AVE': '303 INDEPENDENCE DR',
 '507 EMILY ST': '507 EMILY AVE',
 '614 ROADWAY ST': '614 ROADWAY RD',
 '706 E TOWNE RD': '706 E TOWN RD',
 '719 BARKLEY AVE': '719 BARKLEY BLVD',
 '804 HIGHLAND AVE': '804 W HIGHLAND AVE',
 '919 LINCOLN AVE': '919 W LINCOLN AVE',
 '1210 TENTH AVE': '1210 10TH AVE',
 '1105 ST ANDREWS DR': '1105 SAINT ANDREWS DR',
 '1901 GREENVALE RD': '1901 GREENVALE AVE',
 '2004 WHITNEY AVE': '2004 W WHITNEY AVE',
 '108 INGLESIDE DR': '108 S INGLESIDE DR',
 '1817 GREENVALE RD': '1817 GREENVALE AVE',
 '2116 WADDELL AVE': '2116 W WADDELL AVE',
 '610 LINCOLN AVE': '610 W LINCOLN AVE',
 '1302 E SECOND AVE': '1302 E 2ND AVE',
 '1921 EDGERLY DR': '1921 EDGERLY AVE',
 '202 N W RD': '202 N WEST RD',
 '2020 N RIVERVIEW CR': '2020 N RIVERVIEW CIR',
 '214 N BROADAY ST': '214 N BROADWAY ST',
 '2208 OXFORD DR': '2208 OXFORD RD',
 '502 LINCOLN AVE': '502 W LINCOLN AVE',
 '422 LOIUS AVE': '422 LOUIS AVE',
 '410 MERCER AVE': 'MERCER AVE',
 '1012 W MERCER AVE': '1012 MERCER AVE',
 '1312 MOBILE RD': '1312 MOBILE AVE',
 '1934 S RIVERVIEW CR': '1934 S RIVERVIEW CIR',
 '2028 N RIVERVIEW DR': '2028 N RIVERVIEW CIR',
 '411 BONNYVIEW AVE': '411 BONNY VIEW AVE'}

In [435]:
fixed_addresses = fixed_addresses.replace(dic, regex=True)

In [436]:
fixed_addresses.iloc[471]

'2206 HABERSHAM RDS'

In [438]:
habershams = fixed_addresses[~fixed_addresses.isin(truth)]

In [442]:
for i in habershams.index:
    fixed_addresses.iloc[i] = fixed_addresses.iloc[i][:-1]

In [445]:
df_fixed['Address'] = fixed_addresses

In [446]:
df_fixed['Multihome'] = mh

In [447]:
df_fixed.to_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Test-Replication/Project.csv', index=False)